This project builds an end-to-end machine learning pipeline to predict customer churn using historical customer data. The goal is to identify customers at risk of leaving so retention strategies can be applied proactively.



<mark>add a description here bu note this is supervised training because we are using a target column and features to predict outcome</mark>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

# Load data
df = pd.read_csv("../data/credit_default.csv")

df.head()


Let's understand the data. We will view:
1. size
2. columns (names & data types)
3. summary statistics
4. columns with NULL values

In [ ]:
df.shape

In [ ]:
df.info()
df.columns

In [20]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [ ]:
df.isnull().sum()

Looking at the data,  
Let's ask ourself these questions:
1. What is the target column? 
2. Are most features numeric? 
3. Any obvious ID column?
4. Any weird values?

Note:  
target: The specific column in a dataset that the model is being trained to predict  
features: All the other columns that provide input information used by the model to make its predictions

Observations:  
1. <mark>**default.payment.next.month**</mark> is the target variable  
    * We know this because this variable represents to a future outcome prediction 
    * It represents a binary value (1 for yes and 0 for no)
    * This makes it a **binary classification problem**
    * The task definition:  
    <mark>**Given customer information and payment history, predict whether a customer will default next month.**</mark>

2. All the feature columns are numeric
3. The ID column is explictly defined
4. There all no:  
* NaNs / nulls
* impossible numeric ranges
* corrupted rows  



This dataset is fairly clean 

Let's set the target column **default.payment.next.month**

* default.payment.next.month = 1 → the customer did default  
* default.payment.next.month = 0 → the customer did NOT default


In [ ]:
target_col = "default.payment.next.month"

default.payment.next.month
0    0.7788
1    0.2212
Name: proportion, dtype: float64

Let's answer the questions:  
1. Is the dataset balanced or imbalanced?
2. Roughly what % default vs not?

In [ ]:
# Use value counts to get a new Series object that contains the counts of each unique value
# We are looking for the number of defaults and non defaults 
df[target_col].value_counts()

In [ ]:
# normalize returns percentages instead of raw counts
df[target_col].value_counts(normalize=True)

    The dataset is moderately imbalanced, with the majority of customers not defaulting (78% vs 22%). This makes precision and recall more meaningful than accuracy alone.

Note (Define):  
Precision: Of all the items the model predicted as positive, how many were actually positive?  
Recall: Of all the actual positive items, how many did the model find?  
**Precision** and **recall** are key metrics in evaluating classification models, measuring different aspects of performance

Let's visualize the data:

In [ ]:
sns.countplot(x=target_col, data=df)
plt.title("Target Distribution: Credit Card Default")
plt.show()

Now that we've seen the data distribution,

Let's set our priorities before optimization.

    We understand that ML models aren't perfect and do make mistakes. We must decide how to best to control errors by answering the question:

**Which kind of mistake hurts the business more?**
* false positive (predict default when they won’t)?
* false negative (miss a real default)?

<mark>In this context, false negatives are costly because failing to identify a true defaulter may lead to financial loss. This suggests **recall** may be more important than precision, depending on business strategy. </mark>

Let's remove the ID column because it is non-informative. We don't want the model to accidentally train on the column.

In [27]:
df.drop(columns=["ID"], inplace=True, errors="ignore")
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')